In [1]:
tables=["nbadb_cdm_ab_offer_comm_history_tmp",
"NBADB_CDM_AB_OFFER_COMM_HISTORY",
"nbadb_cdm_ab_contact_history_attr_tmp",
"NBADB_CDM_AB_CONTACT_HISTORY_ATTR",
"nbadb_cdm_ab_contact_history_status_tmp",
"NBADB_CDM_AB_CONTACT_HISTORY_STATUS",
"NBADB_CDM_AB_LOG_BASEOFFER",
"NBADB_CDM_AB_LOG_OFFER",  
"NBADB_CDM_AB_LOG_OFFER_tmp" ,
"NBADB_CDM_AB_RESPONSE_HISTORY", 
"NBADB_CDM_AB_RESPONSE_HISTORY_tmp",
"NBADB_CDM_CI_CONTACT_HISTORY", 
"NBADB_CDM_CI_CONTACT_HISTORY_tmp",   
"NBADB_CDM_AB_RESPONSE_HISTORY_ATTR" , 
"NBADB_CDM_AB_RESPONSE_HISTORY_ATTR_tmp",   
"NBADB_CDM_AB_CONTACT_HISTORY_EXT",   
"NBADB_CDM_AB_CONTACT_HISTORY_EXT_tmp"]


In [3]:
for t in tables:
    print(f"create table t_sourcedata.{t} like s_sourcedata.{t};\n")

create table t_sourcedata.nbadb_cdm_ab_offer_comm_history_tmp like s_sourcedata.nbadb_cdm_ab_offer_comm_history_tmp;

create table t_sourcedata.NBADB_CDM_AB_OFFER_COMM_HISTORY like s_sourcedata.NBADB_CDM_AB_OFFER_COMM_HISTORY;

create table t_sourcedata.nbadb_cdm_ab_contact_history_attr_tmp like s_sourcedata.nbadb_cdm_ab_contact_history_attr_tmp;

create table t_sourcedata.NBADB_CDM_AB_CONTACT_HISTORY_ATTR like s_sourcedata.NBADB_CDM_AB_CONTACT_HISTORY_ATTR;

create table t_sourcedata.nbadb_cdm_ab_contact_history_status_tmp like s_sourcedata.nbadb_cdm_ab_contact_history_status_tmp;

create table t_sourcedata.NBADB_CDM_AB_CONTACT_HISTORY_STATUS like s_sourcedata.NBADB_CDM_AB_CONTACT_HISTORY_STATUS;

create table t_sourcedata.NBADB_CDM_AB_LOG_BASEOFFER like s_sourcedata.NBADB_CDM_AB_LOG_BASEOFFER;

create table t_sourcedata.NBADB_CDM_AB_LOG_OFFER like s_sourcedata.NBADB_CDM_AB_LOG_OFFER;

create table t_sourcedata.NBADB_CDM_AB_LOG_OFFER_tmp like s_sourcedata.NBADB_CDM_AB_LOG_OFFER_tmp;



In [4]:
def s_f1(line):
    print(line[1:-1].split("|"))

def serv_func(line,file):
    tbl,live,deffects,live_l,tbl_l,str_data,test,cleaning,filt,*rest = line[1:-1].lower().split("|")
    print(str_data.replace(" ","") , filt)
    if str_data.replace(" ","") == "структура":
        file.write(f"""beeline -u "jdbc:hive2://bda31node04:10000/;principal=hive/bda31node04.moscow.alfaintra.net@moscow.alfaintra.net" -e "drop table if exists {test}.{tbl_l};"\n""")
        file.write(f"""beeline -u "jdbc:hive2://bda31node04:10000/;principal=hive/bda31node04.moscow.alfaintra.net@moscow.alfaintra.net" -e "create table {test}.{tbl_l} like {live_l}.{tbl_l};"\n""")
    elif str_data.replace(" ","") == "структура+данные" and filt in ["нет", "", " ", "за все время"]:
        file.write(f"""beeline -u "jdbc:hive2://bda31node04:10000/;principal=hive/bda31node04.moscow.alfaintra.net@moscow.alfaintra.net" -e "drop table if exists {test}.{tbl_l};"\n""")
        file.write(f"""beeline -u "jdbc:hive2://bda31node04:10000/;principal=hive/bda31node04.moscow.alfaintra.net@moscow.alfaintra.net" -e "create table {test}.{tbl_l} like {live_l}.{tbl_l};"\n""")
        #file.write(f"""hadoop distcp /live_system/{live_l}/db/{tbl_l}/* /test/{test}/db/{tbl_l}/ \n""")
        file.write(f"""hadoop distcp /storage/{live_l.strip()}/db/{tbl_l}/* /test/{test.strip()}/db/{tbl_l}/ \n""")
        file.write(f"""beeline -u "jdbc:hive2://bda31node04:10000/;principal=hive/bda31node04.moscow.alfaintra.net@moscow.alfaintra.net" -e "msck repair table {test}.{tbl_l};"\n""")
    elif str_data.replace(" ","") == "структура+данные" and filt not in ["нет", "", " "]:
        file.write(f"""beeline -u "jdbc:hive2://bda31node04:10000/;principal=hive/bda31node04.moscow.alfaintra.net@moscow.alfaintra.net" -e "drop table if exists {test}.{tbl_l};"\n""")
        file.write(f"""beeline -u "jdbc:hive2://bda31node04:10000/;principal=hive/bda31node04.moscow.alfaintra.net@moscow.alfaintra.net" -e "create table {test}.{tbl_l} like {live_l}.{tbl_l};"\n""")
        #file.write(f"""hadoop distcp /live_system/{live_l}/db/{tbl_l}/* /test/{test}/db/{tbl_l}/ \n""")
        #file.write(f"""hadoop distcp /storage/{live_l}/db/{tbl_l}/* /test/{test}/db/{tbl_l}/ !!! CUSTOM \n""")
        file.write(f"""beeline -u "jdbc:hive2://bda31node04:10000/;principal=hive/bda31node04.moscow.alfaintra.net@moscow.alfaintra.net" -e "set hive.exec.dynamic.partition=true;set hive.exec.dynamic.partition.mode=nonstrict;set hive.exec.max.dynamic.partitions = 10000;set mapreduce.map.java.opts=-Xmx24G;set mapreduce.map.memory.mb=20480;insert overwrite table {test}.{tbl_l} PARTITION (date_part) select * from {live_l}.{tbl_l} where {filt} ;"\n""")
        file.write(f"""beeline -u "jdbc:hive2://bda31node04:10000/;principal=hive/bda31node04.moscow.alfaintra.net@moscow.alfaintra.net" -e "msck repair table {test}.{tbl_l};"\n""")
with open(r"C:\Users\U_M10Q3\Downloads\act_C52140985_5.txt","r") as f:
    lines=f.readlines()
    resfile=open(r"C:\Users\U_M10Q3\Downloads\act_C52140985_5.sh","a")
    resfile.write("""#!/usr/bin/env bash \n""")
    for line in lines:
        serv_func(line, resfile)
        s_f1(line)
    resfile.close()    
    

структура+данные day>20200101
['MASSAPPSCORINGRESULT', 'l_deriveddata', 'DATALAKE-2309', 'l_hdb_mass_det', 'bapplication', 'Структура+данные', 't_hdb_mass_det', 'Да', 'day>20200101', '']
структура+данные day>20200101
['MASSAPPSCORINGRESULT', 'l_deriveddata', 'DATALAKE-2309', 'l_hdb_mass_det', 'boliverwyman', 'Структура+данные', 't_hdb_mass_det', 'Да', 'day>20200101', '']
структура+данные day>20200101
['MASSAPPSCORINGRESULT', 'l_deriveddata', 'DATALAKE-2309', 'l_hdb_mass_det', 'bresults', 'Структура+данные', 't_hdb_mass_det', 'Да', 'day>20200101', '']
структура  
['deal_edim', 's_dwh4dm', ' ', 's_dwh4dm', 'deal_edim', 'Структура', 't_dwh4dm', 'Да', ' ', '']
структура  
['deal_edim_tmp', 's_dwh4dm', ' ', 's_dwh4dm_tmp', 'deal_edim_tmp', 'Структура', 't_dwh4dm_tmp', 'Да', ' ']


In [ ]:
dmfr="dmfr"
dmfr_tables=["ADJUSTMENTBS_UA_VHIST",
"ADJUSTMENTBS_VHIST",
"ADJUSTMENTPLRB_UA_VHIST",
"ADJUSTMENTPL_UA_VHIST",
"ADJUSTMENTPL_UA_VHIST_TEMP",
"ADJUSTMENTPL_VHIST",
"BSTRANSACTION_HIST_TRAN",
"BSTRANSACTION_TRAN",
"CARDBONUSPOINTS_STRAN",
"CARDTRANSACTION_SAGG",
"CARDTRNINCHNGFEE_STRAN",
"CASHTRANSACTION_TRAN",
"CHECKBALANCE_TRAN",
"DATASETURR_TRAN",
"DEALEXTSRVOPER_STRAN",
"DEALLOANOVRD_SHIST_B",
"DEALLOANOVRDUE_STAT",
"EBUSOPERATION_STRAN",
"EPAYOPERATION_STRAN",
"FRAUDLOAN_TRAN",
"FTPCOMPVALUES_SHIST",
"FTPCOMPVALUES_SHISTOLD",
"IFRSBALANCELOG_TRAN",
"IFRSBALANCE_STAT",
"INSURANCEACCUMINT_STRAN",
"LOANPORTFOLIORTL_TRAN",
"LOANPORTFOLIO_STRAN",
"MCTRANSACTION_STRAN",
"NONLOANPORTFOLIORTL_TRAN",
"ORDERFOREX_STRAN",
"PLTRANSACTION_HIST_TRAN",
"PLTRANSACTION_TRAN",
"PLTRNMARKLOG_TRAN",
"RECOVERYWOIFRS_TRAN",
"SALEDDEBT_TRAN",
"SUBHLDBALANCE_STAT",
"TRNMARKLOG_TRAN",
"VNEBALTRANSACTION_TRAN",
"WRITEOFFRSBU_TRAN"]


In [11]:
import datetime
with  open(r"C:\Users\U_M10Q3\Documents\s_dmcb_cbbalance_stat.txt") as f:
    lines = f.readlines()
    cur_line = datetime.date(2017,1,1)
    start = datetime.date(2017,1,1)
    #f3 = open(r"C:\Distribs\wsrm_lines.txt", "a")
    with open(r"C:\Distribs\s_dmcb_cbbalance_stat_periods.txt", "a") as f2:
        for line in lines:
                    
            line = line[-9:].strip()
            #f3.write(line)
            end = datetime.date(int(line[:4]), int(line[4:6]), int(line[6:]))
            if (end - start).days == 1:
                start = end
            else:    
                print(f"""{cur_line.strftime("%Y%m%d")} - {start.strftime("%Y%m%d")}""" ,file = f2)
                start = end
                cur_line = end
#f3.close()                

In [7]:
(datetime.date(2019,12,12) - datetime.date(2019,12,8)).days

NameError: name 'datetime' is not defined

In [73]:
with  open(r"C:\Users\U_M10Q3\Documents\wsrm_log_parquet.txt") as f:
    lines = f.readlines()
    cur_line = datetime.date(2007,9,16)
    start = datetime.date(2007,9,16)
    #f3 = open(r"C:\Distribs\wsrm_lines.txt", "a")
    
    for line in lines:
        line = line[-9:].strip()
        print(f"""alter table t_hdb_det.wsrm_log_person add if not exists partition (day = {line}) location '/test/t_hdb/DET/WSRM_LOG_PERSON/day={line}';""")

alter table t_hdb_det.wsrm_log_person add if not exists partition (day = 20070916) location '/test/t_hdb/DET/WSRM_LOG_PERSON/day=20070916';
alter table t_hdb_det.wsrm_log_person add if not exists partition (day = 20070917) location '/test/t_hdb/DET/WSRM_LOG_PERSON/day=20070917';
alter table t_hdb_det.wsrm_log_person add if not exists partition (day = 20070918) location '/test/t_hdb/DET/WSRM_LOG_PERSON/day=20070918';
alter table t_hdb_det.wsrm_log_person add if not exists partition (day = 20070919) location '/test/t_hdb/DET/WSRM_LOG_PERSON/day=20070919';
alter table t_hdb_det.wsrm_log_person add if not exists partition (day = 20070920) location '/test/t_hdb/DET/WSRM_LOG_PERSON/day=20070920';
alter table t_hdb_det.wsrm_log_person add if not exists partition (day = 20070921) location '/test/t_hdb/DET/WSRM_LOG_PERSON/day=20070921';
alter table t_hdb_det.wsrm_log_person add if not exists partition (day = 20070922) location '/test/t_hdb/DET/WSRM_LOG_PERSON/day=20070922';
alter table t_hdb_de

alter table t_hdb_det.wsrm_log_person add if not exists partition (day = 20150918) location '/test/t_hdb/DET/WSRM_LOG_PERSON/day=20150918';
alter table t_hdb_det.wsrm_log_person add if not exists partition (day = 20150919) location '/test/t_hdb/DET/WSRM_LOG_PERSON/day=20150919';
alter table t_hdb_det.wsrm_log_person add if not exists partition (day = 20150920) location '/test/t_hdb/DET/WSRM_LOG_PERSON/day=20150920';
alter table t_hdb_det.wsrm_log_person add if not exists partition (day = 20150921) location '/test/t_hdb/DET/WSRM_LOG_PERSON/day=20150921';
alter table t_hdb_det.wsrm_log_person add if not exists partition (day = 20150922) location '/test/t_hdb/DET/WSRM_LOG_PERSON/day=20150922';
alter table t_hdb_det.wsrm_log_person add if not exists partition (day = 20150923) location '/test/t_hdb/DET/WSRM_LOG_PERSON/day=20150923';
alter table t_hdb_det.wsrm_log_person add if not exists partition (day = 20150924) location '/test/t_hdb/DET/WSRM_LOG_PERSON/day=20150924';
alter table t_hdb_de

In [1]:
tables = [
    "DMINHDB_MRM.PFDEVSAMPLE_DMIN",
"DMHDB.PFDEVSAMPLE_STAT",
"DMSHDB.PFPORTFOLIO_DELTA",
"DMHDB.PFPORTFOLIO_STAT",
"DMHDB.V_PFPORTFOLIO",
"DMHDB.V_PFPORTFOLIOCYCLE",
"DMHDB.V_PFPORTFOLIOCYCLE_ADJ",
"DMHDB.V_PFPORTFOLIO_12M",
"DMHDB.V_PFCALCULATIONS",
"DMINHDB_MRM.PFADJUSTMENTS_DMIN",
"DMHDB.PFADJUSTMENTS_STAT",
"DMINHDB_MRM.HVCREDEVSAMPLE_DMIN",
"DMHDB.HVCREDEVSAMPLE_STAT",
"DMSHDB.HVCREPORTFOLIO_DELTA",
"DMHDB.HVCREPORTFOLIO_STAT",
"DMHDB.V_HVCREPORTFOLIO",
"DMHDB.V_HVCREPORTFOLIOCYCLE",
"DMHDB.V_HVCREPORTFOLIOCYCLE_ADJ",
"DMHDB.V_HVCREPORTFOLIO_12M",
"DMHDB.V_HVCRECALCULATIONS",
"DMINHDB_MRM.HVCREADJUSTMENTS_DMIN",
"DMHDB.HVCREADJUSTMENTS_STAT"
]

In [7]:
for t in tables:
    if "DMINHDB_MRM" not in t and "DMSHDB" not in t and ".V_" not in t:
        #print(f"GRANT SELECT ON {t} TO ROL_TECHNOLOGIST;")
        #print(f"GRANT SELECT ON {t} TO ROL_RISK_KB;")
        #print(f"GRANT SELECT ON {t} TO ROL_KB_DATA;")
        print(f"""insert into dbatools.MATRIX_FUNCGROUPOBJECTS values ("{t.split(".")[0]}_ALL_TABLES", "{t.split(".")[0]}", "{t.split(".")[1]}", "TABLE");""")
    else:
     #   print(f"GRANT SELECT ON {t} TO ROL_TECHNOLOGIST;")
        print(f"""insert into dbatools.MATRIX_FUNCGROUPOBJECTS values ("{t.split(".")[0]}_ALL_TABLES", "{t.split(".")[0]}", "{t.split(".")[1]}", "VIEW");""")


insert into dbatools.MATRIX_FUNCGROUPOBJECTS values ("DMINHDB_MRM_ALL_TABLES", "DMINHDB_MRM", "PFDEVSAMPLE_DMIN", "VIEW");
insert into dbatools.MATRIX_FUNCGROUPOBJECTS values ("DMHDB_ALL_TABLES", "DMHDB", "PFDEVSAMPLE_STAT", "TABLE");
insert into dbatools.MATRIX_FUNCGROUPOBJECTS values ("DMSHDB_ALL_TABLES", "DMSHDB", "PFPORTFOLIO_DELTA", "VIEW");
insert into dbatools.MATRIX_FUNCGROUPOBJECTS values ("DMHDB_ALL_TABLES", "DMHDB", "PFPORTFOLIO_STAT", "TABLE");
insert into dbatools.MATRIX_FUNCGROUPOBJECTS values ("DMHDB_ALL_TABLES", "DMHDB", "V_PFPORTFOLIO", "VIEW");
insert into dbatools.MATRIX_FUNCGROUPOBJECTS values ("DMHDB_ALL_TABLES", "DMHDB", "V_PFPORTFOLIOCYCLE", "VIEW");
insert into dbatools.MATRIX_FUNCGROUPOBJECTS values ("DMHDB_ALL_TABLES", "DMHDB", "V_PFPORTFOLIOCYCLE_ADJ", "VIEW");
insert into dbatools.MATRIX_FUNCGROUPOBJECTS values ("DMHDB_ALL_TABLES", "DMHDB", "V_PFPORTFOLIO_12M", "VIEW");
insert into dbatools.MATRIX_FUNCGROUPOBJECTS values ("DMHDB_ALL_TABLES", "DMHDB", "V_PFCA

In [8]:
len(tables)

22

In [9]:
22671-24129

-1458

In [11]:
tact = [
"L_DWH.BALANCE_HSTAT",
"L_DWH4DM.DEAL2ACCTTYPE_LOV" ,
"L_DWH4DM.DEAL2ACCT_SHIST",
"L_DWH4DM.DEALLOANEWSADDINFO_SHIST",
"L_DWH4DM.DEALCLIENTAAMARK_SHIST"
]
for t in tact :
    print(f"drop table if exists t{t[1:]} purge;\n create table t{t[1:]} like {t};")

drop table if exists t_DWH.BALANCE_HSTAT purge;
 create table t_DWH.BALANCE_HSTAT like L_DWH.BALANCE_HSTAT;
drop table if exists t_DWH4DM.DEAL2ACCTTYPE_LOV purge;
 create table t_DWH4DM.DEAL2ACCTTYPE_LOV like L_DWH4DM.DEAL2ACCTTYPE_LOV;
drop table if exists t_DWH4DM.DEAL2ACCT_SHIST purge;
 create table t_DWH4DM.DEAL2ACCT_SHIST like L_DWH4DM.DEAL2ACCT_SHIST;
drop table if exists t_DWH4DM.DEALLOANEWSADDINFO_SHIST purge;
 create table t_DWH4DM.DEALLOANEWSADDINFO_SHIST like L_DWH4DM.DEALLOANEWSADDINFO_SHIST;
drop table if exists t_DWH4DM.DEALCLIENTAAMARK_SHIST purge;
 create table t_DWH4DM.DEALCLIENTAAMARK_SHIST like L_DWH4DM.DEALCLIENTAAMARK_SHIST;


In [13]:
packages(datetime)

NameError: name 'packages' is not defined

In [2]:
import datetime


In [10]:
st=datetime.datetime(2015,5,1)
end = datetime.datetime(2015,9,14)



In [5]:
print(st.strftime("%Y%m%d"))

20150501


In [20]:
st=datetime.datetime(2016,11,18)
end = datetime.datetime(2016,11,27)
while st<=end:
    #st=st.strftime("%Y%m%d")
    print(f"touch flag-{st.strftime('%Y%m%d')}\nhdfs dfs -copyFromLocal flag-{st.strftime('%Y%m%d')} /live_system/l_hdb/mass_daily_insert/flag/\nrm flag-{st.strftime('%Y%m%d')}")
    
    st=st+datetime.timedelta(days=1)
    

touch flag-20161118
hdfs dfs -copyFromLocal flag-20161118 /live_system/l_hdb/mass_daily_insert/flag/
rm flag-20161118
touch flag-20161119
hdfs dfs -copyFromLocal flag-20161119 /live_system/l_hdb/mass_daily_insert/flag/
rm flag-20161119
touch flag-20161120
hdfs dfs -copyFromLocal flag-20161120 /live_system/l_hdb/mass_daily_insert/flag/
rm flag-20161120
touch flag-20161121
hdfs dfs -copyFromLocal flag-20161121 /live_system/l_hdb/mass_daily_insert/flag/
rm flag-20161121
touch flag-20161122
hdfs dfs -copyFromLocal flag-20161122 /live_system/l_hdb/mass_daily_insert/flag/
rm flag-20161122
touch flag-20161123
hdfs dfs -copyFromLocal flag-20161123 /live_system/l_hdb/mass_daily_insert/flag/
rm flag-20161123
touch flag-20161124
hdfs dfs -copyFromLocal flag-20161124 /live_system/l_hdb/mass_daily_insert/flag/
rm flag-20161124
touch flag-20161125
hdfs dfs -copyFromLocal flag-20161125 /live_system/l_hdb/mass_daily_insert/flag/
rm flag-20161125
touch flag-20161126
hdfs dfs -copyFromLocal flag-2016112

In [2]:
import pickle

In [8]:
f = open(r"C:\Distribs\PickleExample", "w+b")
d = {1:"one", 2: "two", 3: [1,2,3]}
pickle.dump(d,f)
f.close()

In [9]:
inf = open(r"C:\Distribs\PickleExample", "r+b")
d1 = pickle.load(inf)
inf.close()

In [10]:
d1

{1: 'one', 2: 'two', 3: [1, 2, 3]}

SyntaxError: invalid syntax (<ipython-input-14-d411d51a0b38>, line 1)

In [4]:
e= " asf "
e.strip()

'asf'

In [8]:
ts={1,"ert"}

In [9]:
ts.add(2)
ts.add(1)
ts

{1, 2, 'ert'}

In [13]:
ts.pop()

KeyError: 'pop from an empty set'

In [3]:
1947-1351 +1 +1

598

In [1]:
import datetime
st=datetime.datetime(2019,11,6)
end = datetime.datetime(2020,1,1)
while st<=end:
    #st=st.strftime("%Y%m%d")
    #print(f"touch flag-{st.strftime('%Y%m%d')}\nhdfs dfs -copyFromLocal flag-{st.strftime('%Y%m%d')} /live_system/l_hdb/mass_daily_insert/flag/\nrm flag-{st.strftime('%Y%m%d')}")
    print(f"{st.strftime('%Y%m%d')}")
    st=st+datetime.timedelta(days=1)

20191106
20191107
20191108
20191109
20191110
20191111
20191112
20191113
20191114
20191115
20191116
20191117
20191118
20191119
20191120
20191121
20191122
20191123
20191124
20191125
20191126
20191127
20191128
20191129
20191130
20191201
20191202
20191203
20191204
20191205
20191206
20191207
20191208
20191209
20191210
20191211
20191212
20191213
20191214
20191215
20191216
20191217
20191218
20191219
20191220
20191221
20191222
20191223
20191224
20191225
20191226
20191227
20191228
20191229
20191230
20191231
20200101


In [4]:
import this


The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!
